In [1]:
import tensorflow as tf
import numpy as np
import glob
import sys
import scipy.misc
from scipy import misc 

In [2]:
def cnn_model(features, labels, mode):
    LEARNING_RATE = 0.001
    DROPOUT = 0.4
    
    input_layer = tf.reshape(features['x'], [-1, 32, 32, 3])
    conv1 = tf.layers.conv2d(inputs= input_layer, filters = 32, kernel_size = 5, padding= "same", activation = tf.nn.relu)
    print("CONV1 SHAPE: ", conv1.shape)
    pool1=  tf.layers.max_pooling2d(inputs = conv1, pool_size = 2, strides = 2) # cuts size in 2
    print("POOL1 SHAPE: ", pool1.shape)
    conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size = 5, padding= "same", activation = tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = 2, strides = 2) # cuts size in 2
    
    
    ## Note: Replace hardcoding in next line with shape of pool2 (minus the batch size)
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense = tf.layers.dense(inputs = pool2_flat, units = 1024, activation = tf.nn.relu)
#     dropout = tf.layers.dropout(inputs = dense, rate = dropout_rate, )
    logits = tf.layers.dense(inputs = dense, units = 10)
    predictions = { "classes": tf.argmax(input = logits, axis = 1),
                    "probabilities": tf.nn.softmax(logits, name = "smt")
                  }
    onehot_labels = tf.one_hot( indices = tf.cast(labels, tf.int32), depth=10 )
    loss = tf.losses.softmax_cross_entropy( onehot_labels= onehot_labels, logits= logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate= LEARNING_RATE)
        train_op = optimizer.minimize(loss = loss, global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)
    else:
        eval_metric_ops = {"accuracy": tf.metrics.accuracy( labels = labels, prediction = prediction["classes"])}
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops = eval_metric_ops)
    

In [3]:
## HELPERS

# Functions to load data, DO NOT change these
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)
        
def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [4]:

# Load the data
data_root_path = '/Users/ravi/Documents/Aalhad/Deep_Learning/HW2/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path)
X_train = X_train.astype(np.float32)
# this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs]
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1)
y_train = np.delete(y_train,indexs)



{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [ ]:
# def main(unused_argv):

#Set hyperparameters
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.4

mnist = tf.contrib.learn.datasets.load_dataset("mnist")

train_data = X_train.T
train_labels = np.array(y_train, dtype = np.int32)
# eval_data = cifar.test.images
# eval_labels = np.asarray(cifar.test.labels, dtype=np.int32)

#estimator
mnist_classifier = tf.estimator.Estimator( model_fn = cnn_model)

#log
tensor_to_log = {"probabilities": "smt"}
logging_hook = tf.train.LoggingTensorHook(
    tensors = tensor_to_log, every_n_iter = 50)

train_input_fn = tf.estimator.inputs.numpy_input_fn( x = {"x": train_data}, y = train_labels, batch_size = 100, num_epochs = None, shuffle = True)
mnist_classifier.train(input_fn = train_input_fn, steps = 20000, hooks = [])

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/2x/7scwzsy54_b82kjky8p3nn780000gn/T/tmp3xmol6ri', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
CONV1 SHAPE:  (100, 32, 32, 32)
POOL1 SHAPE:  (100, 16, 16, 32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/2x/7scwzsy54_b82kjky8p3nn780000gn/T/tmp3xmol6ri/model.ckpt.
INFO:tensorflow:loss = 2.33339, step = 1
INFO:tensorflow:global_step/sec: 2.18621
INFO:tensorflow:loss = 2.31581, step = 101 (45.743 sec)
INFO:tensorflow:global_step/sec: 2.0887
INFO:tensorflow:l